In [1]:
import pandas as pd
import polars as pl
from great_tables import GT

In [63]:
def direct_proportion_normalize(series, direction='positive'):
            """
            Normaliza una serie de datos usando el método de Proporción Directa (Normalización a la Suma).
            Implementa un 'shift' para asegurar que todos los valores sean no negativos.
            
            - Si la dirección es 'positive' (Alto=Bueno), usa Proporción Directa.
            - Si la dirección es 'negative' (Alto=Malo), usa una Proporción Inversa Suavizada 
            (penalizando valores altos) basada en la fórmula R/(R+X) donde R es la media + std.
            """
            
            # Asegurar que el input es una Serie de Pandas
            if not isinstance(series, pd.Series):
                series = pd.Series(series)

            # 1. SHIFTING: Asegurar que el valor mínimo de la serie sea 0 o positivo.
            min_val = series.min()
            
            # Shift para que el mínimo sea exactamente 0 (si era negativo)
            if min_val < 0:
                # Sumar el valor absoluto del mínimo a toda la serie.
                shifted_series = series - min_val 
            else:
                # No se necesita shift si el mínimo es 0 o positivo.
                shifted_series = series
            
            # --- Lógica de Normalización ---
            
            if direction == 'positive':
                # Alto=Bueno (Proporción Directa Estándar)
                total_sum = shifted_series.sum()
                
                if total_sum == 0:
                    # Caso extremo: si todos los valores son cero después del shift.
                    return pd.Series(1.0 / len(shifted_series), index=shifted_series.index)
                
                return shifted_series / total_sum
            
            elif direction == 'negative':
                
                # Alto=Malo: Proporción Inversa Suavizada (penalizando valores altos)
                
                # 1. Definir la constante R como el punto de referencia para la inversión suave.
                # R = Media + Desviación Estándar (evita el efecto extremo de la inversa 1/X)
                R = series.mean() + series.std()*3

                # Manejo del caso extremo R=0
                if R == 0:
                    # Si todos los valores de la serie shiftada son cero (o idénticos, resultando en std=0)
                    # Asignar distribución equitativa.
                    return pd.Series(1.0 / len(series), index=series.index)

                # 2. Aplicar la función de penalización: R / (R + X)
                # Un valor grande (Alto=Malo) dará un resultado penalizado (cercano a 0).
                penalized_series = R + series
                
                # 3. Normalizar la serie penalizada a la suma (Proporción Directa)
                total_sum_penalized = penalized_series.sum()
                
                if total_sum_penalized == 0:
                    # Si, por alguna razón, la suma es 0 después de la penalización (muy improbable con R>0)
                    return pd.Series(1.0 / len(series), index=series.index)
                    
                return penalized_series / total_sum_penalized


In [35]:
df = pd.read_csv('fasp_datos_entrada.csv')

In [36]:
df.columns

Index(['Entidad_Federativa', 'Pob', 'Var_inc_del', 'Tasa_policial', 'Dig_salarial', 'Profesionalizacion', 'Ctrl_conf', 'Disp_camaras', 'Disp_lectores_veh',
       'Tasa_abandono_llamadas', 'Cump_presup', 'Sobrepob_penitenciaria', 'Proc_justicia', 'Servs_forenses', 'Eficiencia_procesal', 'Asignacion_2025'],
      dtype='object')

In [64]:
direct_proportion_normalize(df['Dig_salarial'], direction='positive')

0     0.031376
1     0.045579
2     0.013195
3     0.010305
4     0.025163
5     0.019943
6     0.021631
7     0.028935
8     0.027990
9     0.027229
10    0.083487
11    0.014642
12    0.027988
13    0.038269
14    0.025377
15    0.010686
16    0.033702
17    0.029794
18    0.045165
19    0.029792
20    0.039052
21    0.059161
22    0.014619
23    0.062803
24    0.025811
25    0.028581
26    0.000000
27    0.023854
28    0.011365
29    0.084135
30    0.039594
31    0.020779
Name: Dig_salarial, dtype: float64